In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle as pkl
import re

In [2]:
# Routines to calculate the number of days between two dates. Useful for data preprocessing
def convert_str_date_to_ints(date1, date2):
    # Date is formatted as "yyyy-mm-dd"
    [y1, m1, d1] = [int(elem) for elem in re.split('_|-', date1)]
    [y2, m2, d2] = [int(elem) for elem in re.split('_|-', date2)]
    if y1 > y2:
        return  (y2, m2, d2), (y1, m1, d1)
    elif y1 < y2:
        return  (y1, m1, d1), (y2, m2, d2)
    
    if m1 > m2:
        return  (y2, m2, d2), (y1, m1, d1)
    elif m1 < m2:
        return  (y1, m1, d1), (y2, m2, d2)
    
    if d1 > d2:
        return  (y2, m2, d2), (y1, m1, d1)
    
    return  (y1, m1, d1), (y2, m2, d2)
    
    
def is_leap(year):
    if (year % 4 == 0):
        if (year % 100 != 0):
            return True
        else:
            return (year % 400 == 0)
    return False


def get_number_of_days(y, m):
    if m in {1,3,5,7,8,10,12}:
        return 31
    elif m in {4,6,9,11}:
        return 30
    else:
        return 29 if  is_leap(y) else 28

def daysBetweenDates(y1, m1, d1, y2, m2, d2) -> int:
    # Assumes y2 >= y1
    days = 0
    # Years
    for y in range(y1 + 1, y2):
        for m in range(1, 13):
            days += get_number_of_days(y, m)
    # Months
    if y1 < y2:
        for m in range(m1 + 1, 13):
            days += get_number_of_days(y1, m)

        for m in range(1, m2):
            days += get_number_of_days(y2, m)

    else:
        for m in range(m1 + 1, m2):
            days += get_number_of_days(y1, m)


    # Days
    if y1 == y2 and m1 == m2:
        days += d2 - d1
    else:
        days += get_number_of_days(y1, m1) - d1  + d2

    return days

# This function returns chunks of continuous times in which there are no NaNs... In the event there is a NaN in the middle, the chunks are split in two halfs
# fragmented.
def get_chunks(df, location_key, locations, features, output_dic = "./dic.pkl"):
    global_dic = {}
    
    # Load the dataframe
    #df = pd.read_csv(df_path)
    
    # Process countries
    for location in locations:
        loc_df = df[df[location_key].isin([location])]
        global_dic[location] = {}
        
        # Process features
        for feat in features:
            #feat_df = loc_df.loc[loc_df.columns[loc_df.columns.isin([feat, location_key, 'date'])]]
            feat_df = loc_df[loc_df.columns[loc_df.columns.isin([feat, location_key, 'date'])]]

            prev_date, global_dic[location][feat] = None, []
            curr_chunk, curr_date = [], None
            
            if len(feat_df.columns) != 3:
                continue
            
            feat_df = feat_df.dropna()
            if len(feat_df) == 0:
                continue            

            for index, row in feat_df.iterrows():
                curr_date = row['date']
                if prev_date is None:
                    prev_date = curr_date
                    curr_chunk.append(prev_date)
                    continue
                date1, date2 = convert_str_date_to_ints(prev_date, curr_date)
                diff = daysBetweenDates(*date1, *date2)
                if diff > 1:
                    curr_chunk.append(prev_date)
                    global_dic[location][feat].append(curr_chunk)
                    curr_chunk = [curr_date]
                prev_date = curr_date
            if curr_date is not None:   
                curr_chunk.append(curr_date)
                if len(curr_chunk) == 1:
                    curr_chunk.append(curr_date)
                global_dic[location][feat].append(curr_chunk)
    global_dic = compute_effective_range(global_dic)
    with open(output_dic, "wb") as fp:
        pkl.dump(global_dic, fp)
    return global_dic

def cut_down_dates(df, global_dic):
    for country_name, value in global_dic.items():
        start_date, end_date = value['effective_range'][0], value['effective_range'][1]
        # Truncate to the effective range
        index = df.loc[(df['location'] == country_name) & ((df['date'] < start_date) | (df['date'] > end_date))].index
        df = df.drop(index , inplace=False)
    return df



def filter_sets(reference, dropouts):
    new_set = set()
    for elem in reference:
        if elem not in dropouts:
            new_set.add(elem)
    return new_set
    

    
def compute_effective_range(dic):
    for country, entries in dic.items():
        min_date, max_date = '0000-01-01', '3000-12-31' 
        for v_list in entries.values():
            if not v_list:
                continue
            start, end = v_list[0][0], v_list[-1][-1]
            if start > min_date:
                min_date = start
            if end < max_date:
                max_date = end
        if min_date != '0000-01-01':
            dic[country]['effective_range'] = [min_date, max_date]
    return dic

## Paris

In [43]:
df_rec = pd.read_csv(f'./cities/raw/paris_rec.csv', low_memory=False).drop(
        ['granularite', 'cas_confirmes', 'hospitalises', 'maille_code', 'cas_ehpad', 'depistes', 'source_nom', 'nouvelles_hospitalisations', 'reanimation', 'cas_confirmes_ehpad',
         'deces', 'nouvelles_reanimations', 'source_url', 'source_archive', 'source_type', 'depistes', 'deces_ehpad', 'cas_possibles_ehpad'], axis=1)
df_rec = df_rec.rename(columns={
  "maille_nom":"location",
  "gueris":"cum_recovered",
})
df_rec = df_rec.loc[(df_rec['location'] == 'Paris')]
df_rec['location'] = 'paris'
start_date = '2020-03-18'
# Truncate to the effective range
index = df_rec.loc[(df_rec['date'] < start_date)].index
df_rec = df_rec.drop(index , inplace=False)
df_rec

,date,location,cum_recovered
2126,2020-03-18,paris,40.0
2354,2020-03-19,paris,62.0
2572,2020-03-20,paris,98.0
2802,2020-03-21,paris,122.0
3031,2020-03-22,paris,145.0
...,...,...,...
64529,2021-08-08,paris,21128.0
64649,2021-08-09,paris,21150.0
64769,2021-08-10,paris,21167.0
64889,2021-08-11,paris,21189.0


In [44]:
df_no_rec = pd.read_csv(f'./cities/raw/paris_no_rec.csv', low_memory=False).drop(
        ['id',  'tests', 'people_vaccinated', 'icu', 'vent', 'school_closing', 'workplace_closing', 'cancel_events',
         'gatherings_restrictions', 'transport_closing','stay_home_restrictions', 'internal_movement_restrictions',
         'international_movement_restrictions', 'information_campaigns', 'testing_policy', 'contact_tracing', 
         'facial_coverings', 'vaccination_policy', 'elderly_people_protection', 'government_response_index', 
         'stringency_index', 'containment_health_index', 'economic_support_index', 'administrative_area_level', 
         'administrative_area_level_1',  'administrative_area_level_2', "administrative_area_level_3",  'latitude', 'longitude', 'iso_alpha_3', 
         'iso_alpha_2',  'iso_numeric', 'iso_currency', 'key_local', 'key_google_mobility', 'key_jhu_csse', 'key_nuts', 
         'key_gadm', 'vaccines', 'recovered'], axis=1)
df_no_rec = df_no_rec.rename(columns={
  "key_apple_mobility":"location",
  "confirmed":"cum_confirmed", 
  "deaths":"cum_deceased", 
  "hosp":"current_hospitalized_patients", 
  "people_fully_vaccinated":"cum_persons_vaccinated"
})
df_no_rec['location'] = 'paris'
end_date = '2021-08-12'
# Truncate to the effective range
index = df_no_rec.loc[(df_no_rec['date'] > end_date)].index
df_no_rec = df_no_rec.drop(index , inplace=False)
df_no_rec

,date,cum_confirmed,cum_deceased,cum_persons_vaccinated,current_hospitalized_patients,population,location
0,2020-03-18,NaN,14.0,NaN,359.0,2148271,paris
1,2020-03-19,NaN,22.0,NaN,453.0,2148271,paris
2,2020-03-20,NaN,30.0,NaN,575.0,2148271,paris
3,2020-03-21,NaN,36.0,NaN,649.0,2148271,paris
4,2020-03-22,NaN,40.0,NaN,728.0,2148271,paris
...,...,...,...,...,...,...,...
508,2021-08-08,268458.0,4610.0,1378257.0,278.0,2148271,paris
509,2021-08-09,269186.0,4614.0,1386705.0,285.0,2148271,paris
510,2021-08-10,269825.0,4617.0,1395341.0,284.0,2148271,paris
511,2021-08-11,270446.0,4620.0,1403040.0,279.0,2148271,paris


In [46]:
df_consolidated = pd.merge(df_no_rec, df_rec[['date','cum_recovered']], on='date')
features = {'cum_deceased', 'current_hospitalized_patients', 'cum_recovered'}
chunks = get_chunks(df_consolidated, "location", ['paris'], features, output_dic = f'./cities/preprocessed/{city}-chunk.pkl')
df_consolidated = cut_down_dates(df_consolidated, chunks)
df_consolidated = df_consolidated.interpolate(method='linear', limit_direction='both')
print(f"################### CITY OF Paris: #NaNs after preprocessing == {df_consolidated.isna().sum().sum()}")
df_consolidated.to_csv(f"./cities/preprocessed/paris.csv")

################### CITY OF Paris: #NaNs after preprocessing == 0


## New York

In [ ]:
city = 'new-york'
df = pd.read_csv(f'./cities/raw/{city}.csv', low_memory=True).drop(
    ['id',  'tests', 'people_vaccinated', 'icu', 'vent', 'school_closing', 'workplace_closing', 'cancel_events',
     'gatherings_restrictions', 'transport_closing','stay_home_restrictions', 'internal_movement_restrictions',
     'international_movement_restrictions', 'information_campaigns', 'testing_policy', 'contact_tracing', 
     'facial_coverings', 'vaccination_policy', 'elderly_people_protection', 'government_response_index', 
     'stringency_index', 'containment_health_index', 'economic_support_index', 'administrative_area_level', 
     'administrative_area_level_1',  'administrative_area_level_2', "administrative_area_level_3",  'latitude', 'longitude', 'iso_alpha_3', 
     'iso_alpha_2',  'iso_numeric', 'iso_currency', 'key_local', 'key_google_mobility', 'key_jhu_csse', 'key_nuts', 
     'key_gadm', 'vaccines'], axis=1)
df = df.rename(columns={
  "key_apple_mobility":"location",
  "confirmed":"cum_confirmed", 
  "deaths":"cum_deceased", 
  "hosp":"current_hospitalized_patients", 
  "recovered":"cum_recovered",
  "people_fully_vaccinated":"cum_persons_vaccinated"
})
print(f"################### CITY OF '{city.upper()}': #NaNs before preprocessing == {df.isna().sum().sum()}")

for col in df.columns:
    print(f"===>>> Number of NaN for column '{col}' = {df[col].isna().sum()} (out of {len(df)} values)")

In [7]:
city = 'new-york_rec'
df = pd.read_csv(f'./cities/raw/{city}.csv', low_memory=False).drop(
    ['death', 'deathConfirmed', 'deathIncrease',
       'deathProbable', 'hospitalized', 'hospitalizedCumulative',
       'hospitalizedCurrently', 'hospitalizedIncrease', 'inIcuCumulative',
       'inIcuCurrently', 'negative', 'negativeIncrease',
       'negativeTestsAntibody', 'negativeTestsPeopleAntibody',
       'negativeTestsViral', 'onVentilatorCumulative', 'onVentilatorCurrently',
       'positive', 'positiveCasesViral', 'positiveIncrease', 'positiveScore',
       'positiveTestsAntibody', 'positiveTestsAntigen',
       'positiveTestsPeopleAntibody', 'positiveTestsPeopleAntigen',
       'positiveTestsViral', 'totalTestEncountersViral',
       'totalTestEncountersViralIncrease', 'totalTestResults',
       'totalTestResultsIncrease', 'totalTestsAntibody', 'totalTestsAntigen',
       'totalTestsPeopleAntibody', 'totalTestsPeopleAntigen',
       'totalTestsPeopleViral', 'totalTestsPeopleViralIncrease',
       'totalTestsViral', 'totalTestsViralIncrease'], axis=1)
df = df.rename(columns={
  "state":"location",
  "confirmed":"cum_confirmed", 
  "deaths":"cum_deceased", 
  "hosp":"current_hospitalized_patients", 
  "recovered":"cum_recovered",
  "people_fully_vaccinated":"cum_persons_vaccinated"
})
df

,date,location,cum_recovered
0,2021-03-07,NY,NaN
1,2021-03-06,NY,NaN
2,2021-03-05,NY,NaN
3,2021-03-04,NY,NaN
4,2021-03-03,NY,NaN
...,...,...,...
366,2020-03-06,NY,NaN
367,2020-03-05,NY,NaN
368,2020-03-04,NY,NaN
369,2020-03-03,NY,NaN


In [10]:
city = 'new-york'
df = pd.read_csv(f'./cities/raw/{city}.csv', low_memory=False)
df

,id,date,confirmed,deaths,recovered,tests,vaccines,people_vaccinated,people_fully_vaccinated,hosp,...,iso_alpha_3,iso_alpha_2,iso_numeric,iso_currency,key_local,key_google_mobility,key_apple_mobility,key_jhu_csse,key_nuts,key_gadm
0,bae2006a,2020-03-01,1.0,0.0,NaN,4,NaN,NaN,NaN,NaN,...,USA,US,840,USD,36,ChIJqaUj8fBLzEwRZ5UY3sHGz90,New York,US36,NaN,USA.33_1
1,bae2006a,2020-03-02,1.0,0.0,NaN,11,NaN,NaN,NaN,NaN,...,USA,US,840,USD,36,ChIJqaUj8fBLzEwRZ5UY3sHGz90,New York,US36,NaN,USA.33_1
2,bae2006a,2020-03-03,2.0,0.0,NaN,33,NaN,NaN,NaN,NaN,...,USA,US,840,USD,36,ChIJqaUj8fBLzEwRZ5UY3sHGz90,New York,US36,NaN,USA.33_1
3,bae2006a,2020-03-04,11.0,0.0,NaN,143,NaN,NaN,NaN,NaN,...,USA,US,840,USD,36,ChIJqaUj8fBLzEwRZ5UY3sHGz90,New York,US36,NaN,USA.33_1
4,bae2006a,2020-03-05,22.0,0.0,NaN,302,NaN,NaN,NaN,NaN,...,USA,US,840,USD,36,ChIJqaUj8fBLzEwRZ5UY3sHGz90,New York,US36,NaN,USA.33_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,bae2006a,2024-02-14,NaN,NaN,NaN,108173419,NaN,NaN,NaN,803.0,...,USA,US,840,USD,36,ChIJqaUj8fBLzEwRZ5UY3sHGz90,New York,US36,NaN,USA.33_1
1446,bae2006a,2024-02-15,NaN,NaN,NaN,108183455,NaN,NaN,NaN,781.0,...,USA,US,840,USD,36,ChIJqaUj8fBLzEwRZ5UY3sHGz90,New York,US36,NaN,USA.33_1
1447,bae2006a,2024-02-16,NaN,NaN,NaN,108193030,NaN,NaN,NaN,719.0,...,USA,US,840,USD,36,ChIJqaUj8fBLzEwRZ5UY3sHGz90,New York,US36,NaN,USA.33_1
1448,bae2006a,2024-02-17,NaN,NaN,NaN,108199367,NaN,NaN,NaN,697.0,...,USA,US,840,USD,36,ChIJqaUj8fBLzEwRZ5UY3sHGz90,New York,US36,NaN,USA.33_1


In [11]:
df['recovered'].isna().sum()

1450

## Singapore

In [3]:
city = 'singapore'
df = pd.read_csv(f'./cities/raw/{city}.csv', low_memory=True).drop(
    ['id',  'tests', 'people_vaccinated', 'icu', 'vent', 'school_closing', 'workplace_closing', 'cancel_events',
     'gatherings_restrictions', 'transport_closing','stay_home_restrictions', 'internal_movement_restrictions',
     'international_movement_restrictions', 'information_campaigns', 'testing_policy', 'contact_tracing', 
     'facial_coverings', 'vaccination_policy', 'elderly_people_protection', 'government_response_index', 
     'stringency_index', 'containment_health_index', 'economic_support_index', 'administrative_area_level', 
     'administrative_area_level_1',  'administrative_area_level_2', "administrative_area_level_3",  'latitude', 'longitude', 'iso_alpha_3', 
     'iso_alpha_2',  'iso_numeric', 'iso_currency', 'key_local', 'key_google_mobility', 'key_jhu_csse', 'key_nuts', 
     'key_gadm', 'vaccines'], axis=1)
df = df.rename(columns={
  "key_apple_mobility":"location",
  "confirmed":"cum_confirmed", 
  "deaths":"cum_deceased", 
  "hosp":"current_hospitalized_patients", 
  "recovered":"cum_recovered",
  "people_fully_vaccinated":"cum_persons_vaccinated"
})
print(f"################### CITY OF '{city.upper()}': #NaNs before preprocessing == {df.isna().sum().sum()}")

for col in df.columns:
    print(f"===>>> Number of NaN for column '{col}' = {df[col].isna().sum()} (out of {len(df)} values)")

################### CITY OF 'SINGAPORE': #NaNs before preprocessing == 723
===>>> Number of NaN for column 'date' = 0 (out of 731 values)
===>>> Number of NaN for column 'cum_confirmed' = 1 (out of 731 values)
===>>> Number of NaN for column 'cum_deceased' = 59 (out of 731 values)
===>>> Number of NaN for column 'cum_recovered' = 215 (out of 731 values)
===>>> Number of NaN for column 'cum_persons_vaccinated' = 359 (out of 731 values)
===>>> Number of NaN for column 'current_hospitalized_patients' = 89 (out of 731 values)
===>>> Number of NaN for column 'population' = 0 (out of 731 values)
===>>> Number of NaN for column 'location' = 0 (out of 731 values)


In [52]:
for city in ['singapore', 'new-york', 'tokyo']:
df = pd.read_csv(f'./cities/raw/{city}.csv', low_memory=True).drop(
    ['id',  'tests', 'people_vaccinated', 'icu', 'vent', 'school_closing', 'workplace_closing', 'cancel_events',
     'gatherings_restrictions', 'transport_closing','stay_home_restrictions', 'internal_movement_restrictions',
     'international_movement_restrictions', 'information_campaigns', 'testing_policy', 'contact_tracing', 
     'facial_coverings', 'vaccination_policy', 'elderly_people_protection', 'government_response_index', 
     'stringency_index', 'containment_health_index', 'economic_support_index', 'administrative_area_level', 
     'administrative_area_level_1',  'administrative_area_level_2', "administrative_area_level_3",  'latitude', 'longitude', 'iso_alpha_3', 
     'iso_alpha_2',  'iso_numeric', 'iso_currency', 'key_local', 'key_google_mobility', 'key_jhu_csse', 'key_nuts', 
     'key_gadm', 'vaccines'], axis=1)
df = df.rename(columns={
  "key_apple_mobility":"location",
  "confirmed":"cum_confirmed", 
  "deaths":"cum_deceased", 
  "hosp":"current_hospitalized_patients", 
  "recovered":"cum_recovered",
  "people_fully_vaccinated":"cum_persons_vaccinated"
})
df['location'] = city
features = {'cum_deceased', 'current_hospitalized_patients', 'cum_recovered'}
chunks = get_chunks(df, "location", [city], features, output_dic = f'./cities/preprocessed/{city}-chunk.pkl')
#print(f"Effective range for {city} city: {chunks[city]['effective_range']}")
df = cut_down_dates(df, chunks)
df = df.interpolate(method='linear', limit_direction='both')
df = df.sort_values(by=['date'])
print(f"################### CITY OF '{city.upper()}': Length of dataset == {len(df)}")
df.to_csv(f'./cities/preprocessed/{city}.csv')
for col in df.columns:
    print(f"===>>> Number of NaN for column '{col}' = {df[col].isna().sum()}")

################### CITY OF 'SINGAPORE': Length of dataset == 472
===>>> Number of NaN for column 'date' = 0
===>>> Number of NaN for column 'cum_confirmed' = 0
===>>> Number of NaN for column 'cum_deceased' = 0
===>>> Number of NaN for column 'cum_recovered' = 0
===>>> Number of NaN for column 'cum_persons_vaccinated' = 0
===>>> Number of NaN for column 'current_hospitalized_patients' = 0
===>>> Number of NaN for column 'population' = 0
===>>> Number of NaN for column 'location' = 0
################### CITY OF 'NEW-YORK': Length of dataset == 1077
===>>> Number of NaN for column 'date' = 0
===>>> Number of NaN for column 'cum_confirmed' = 0
===>>> Number of NaN for column 'cum_deceased' = 0
===>>> Number of NaN for column 'cum_recovered' = 1077
===>>> Number of NaN for column 'cum_persons_vaccinated' = 0
===>>> Number of NaN for column 'current_hospitalized_patients' = 0
===>>> Number of NaN for column 'population' = 0
===>>> Number of NaN for column 'location' = 0
###################

In [16]:
df

,date,cum_confirmed,cum_deceased,cum_recovered,vaccines,cum_persons_vaccinated,current_hospitalized_patients,population,location
51,2020-05-08,4810,171.0,2136.0,NaN,NaN,2503.0,13942856,tokyo
52,2020-05-09,4846,180.0,2152.0,NaN,NaN,2514.0,13942856,tokyo
53,2020-05-10,4868,180.0,2170.0,NaN,NaN,2518.0,13942856,tokyo
54,2020-05-11,4883,189.0,2185.0,NaN,NaN,2509.0,13942856,tokyo
55,2020-05-12,4987,196.0,2901.0,NaN,NaN,1890.0,13942856,tokyo
...,...,...,...,...,...,...,...,...,...
1140,2023-05-03,4382724,8111.0,3093983.0,NaN,NaN,47585.0,13942856,tokyo
1141,2023-05-04,4383630,8114.0,3093983.0,NaN,NaN,47592.0,13942856,tokyo
1142,2023-05-05,4384692,8117.0,3093983.0,NaN,NaN,47599.0,13942856,tokyo
1143,2023-05-06,4387037,8120.0,3093983.0,NaN,NaN,47562.0,13942856,tokyo
